In [ ]:
import pandas as pd
df = pd.read_csv("../../data/annonces-ventes-69.csv",delimiter=';',header=0,encoding="UTF-8",low_memory=False)
print (df.shape)
# display(df.head(20))
df = df [df.typedebien == "a"]
print (df.info())


In [ ]:
# drop empy columns
print (df.shape)
# threshold = 90%
threshold = int(df.shape[0]*0.9)
empty_cols = []
for name,_ in df.items() :
    if df[name].isna().sum() > threshold :
        empty_cols.append(name) 
print (f"empty cols {empty_cols}")
# remove empty cols
df = df.drop(columns=empty_cols)
print(df.isna().sum())
# remove unnecessary columns
columns = ["typedebien","typedetransaction","mensualiteFinance","nb_terraces","REG","DEP"]
df = df.drop(columns=columns)
# check for columns content
for column in df.columns:
    print(f"{column}: {df[column].unique()}")
columns = ["type_annonceur","typedebien_lite",'duree_int']
df = df.drop(columns=columns)

In [ ]:
#keep rows with no Nans
#keep rows with no Nans on these attributes

subset = ["chauffage_mode","chauffage_systeme","chauffage_energie","annee_construction","cave","charges_copro","ascenseur"]
df= df.dropna(subset=subset)
print(df.shape)
df.isna().sum()
df.info()

for column in df.columns:
    print(f"{column}: {df[column].unique()}")

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         display(df.head(100))


In [ ]:
import numpy as np

# clean column nb_logements_copro
df["nb_logements_copro"] = df["nb_logements_copro"].fillna(df["nb_logements_copro"].mean()).astype("int")
df["nb_logements_copro"].unique()

# clean anne construction 
df["annee_construction"] = df["annee_construction"].fillna(df["annee_construction"].mean())
df["annee_construction"] = df["annee_construction"].astype("int")
df["annee_construction"].unique()

# places parking , set nan to zero
df["places_parking"] = df["places_parking"].fillna(0)
df["places_parking"] = df["places_parking"].astype("int")
df["places_parking"].unique()

# dpe 
df["dpeL"] = df["dpeL"].replace(to_replace=["D - 231 kWh/m².year","D-218 kWh/m².year","Blank"],value=["D","D","U"])
df["dpeL"].unique()

# replace Électricité by Elec
print(df["chauffage_energie"].unique())
df["chauffage_energie"] = df["chauffage_energie"].replace(to_replace=["Électrique"],value=["Elec"])
print(df["chauffage_energie"].value_counts())

# gaz a effet de serre : set to unknown
print(df["ges_class"].isna().sum())
df["ges_class"] = df["ges_class"].fillna("U")
print(df["ges_class"].value_counts())

#  clarify exposition
df["exposition"] = df["exposition"].replace(to_replace=["0"],value=["Unknown"])
df["exposition"] = df["exposition"].replace(to_replace=["N","NO","NS","Nord-Sud","Nord Est Ouest"],value=["Nord","Nord-Ouest","Nord","Nord","Nord-Est"])
df["exposition"] = df["exposition"].replace(to_replace=["SO"],value=["Sud-Ouest"])
df["exposition"] = df["exposition"].replace(to_replace=["Sud-Est-Ouest"],value=["Sud-Est"])
df["exposition"] = df["exposition"].replace(to_replace=["Est-Ouest"],value=["Est"])
print(df["exposition"].value_counts())

df["nb_toilettes"] = df["nb_toilettes"].fillna(1).astype(int)
print (df["nb_toilettes"].isna().sum())

df["dpeC"] = df["dpeC"].fillna(0)
print (df["dpeC"].isna().sum())

nb_etage = df['nb_etages'].value_counts().index[0]
df["nb_etages"] = df["nb_etages"].fillna(nb_etage).astype(int)
print(f" etages {df['nb_etages'].value_counts()}")

print (df['porte_digicode'].isna().sum())
df["porte_digicode"] = df["porte_digicode"].fillna(False)

attributes = ["loyer_m2_median_n6","nb_log_n6","taux_rendement_n6"]
for attribute in attributes :
    df[attribute]= df[attribute].fillna(0)
    
attributes = ["loyer_m2_median_n7","nb_log_n7","taux_rendement_n7"]
for attribute in attributes :
    df[attribute]= df[attribute].fillna(0)

# charges copro / remove spurious value
df['charges_copro'] = df['charges_copro'].apply(lambda x : df['charges_copro'].mean()  if x > 8000 else x)
print (df['charges_copro'].max())
print (f" {df['charges_copro'].min()} {df['charges_copro'].max()} ")
bins = np.arange(0,8000,250).tolist()
labels = [str(x) for x in bins]
labels.pop()
df['charges_copro_cat']=pd.cut(x = df['charges_copro'],bins=bins,labels=labels,include_lowest = True)

#year of construction
df['annee_construction'] = df['annee_construction'].apply(lambda x : df['annee_construction'].mean()  if x <1900  else x)
print (df['annee_construction'].min())
bins = np.arange(1900,2030,10).tolist()
labels = [str(x) for x in bins]
labels.pop(0)
df['annee_construction_cat']=pd.cut(x = df['annee_construction'],bins=bins,labels=labels,include_lowest = True)

# surface / remove spurious value
df['surface'] = df['surface'].apply(lambda x : 250  if x > 250 else x)
print (df['surface'].max())
print (f" {df['surface'].min()} {df['surface'].max()} ")
bins = np.arange(0,250,10).tolist()
labels = [str(x) for x in bins]
labels.pop()
df['surface_cat']=pd.cut(x = df['surface'],bins=bins,labels=labels,include_lowest = True)

df['nb_logements_copro'] = df['nb_logements_copro'].apply(lambda x : 1010  if x >1000  else x)
bins = np.arange(0,1000,20).tolist()
labels = [str(x) for x in bins]
labels.pop()
df['nb_logements_copro_cat']=pd.cut(x = df['nb_logements_copro'],bins=bins,labels=labels,include_lowest = True)


In [ ]:
df.isna().sum()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df.sort_values(by="prix_m2_vente",ascending=True)
columns = df.columns
print (type(columns))
fig , axes = plt.subplots(nrows = 12, ncols = 4, figsize = (25,50))
index = 0
for column in columns :
    if column not in ["idannonce","date"] :
        sns.scatterplot(ax=axes[int(index/4),index%4],y=df["prix_m2_vente"],x=df[column])
        index = index +1

        

In [ ]:
print (df.columns)

ncols=3
fig , axes = plt.subplots(nrows = 11, ncols = ncols, figsize = (25,50))
index = 0
columns = ["exposition","ges_class","cave",'chauffage_energie',
       'chauffage_systeme', 'chauffage_mode','logement_neuf','dpeL',"etage","nb_etages","balcon","ascenseur","annee_construction_cat","surface_cat","charges_copro_cat","nb_logements_copro_cat","INSEE_COM"]
for column in columns :
    sns.boxplot(ax=axes[int(index/ncols),index%ncols],y=df["prix_m2_vente"],x=df[column])
    index = index +1
